In [268]:
from torch import nn
from torch.nn import functional as F
import torch
from sklearn import metrics

class BagModel_3d(nn.Module):
    '''
    BagModel_3d - not scalable, used with data represented in 3d
    Accepts 3d data tensor and n_instances array
    '''
    def __init__(self, prepNN, afterNN, aggregation_func):
        super().__init__()
        
        self.prepNN = prepNN
        self.afterNN = afterNN
        self.aggregation_func = aggregation_func

#     @profile
    def forward(self, input, n_instances):
        
        NN_out = self.prepNN(input) # Forward all indices through neural network
        
        output = torch.empty(size = (input.size(0), len(NN_out[0][0])), dtype = torch.double) # Pre-alocate tensor for output

        for i, n in enumerate(n_instances):
            output[i] = self.aggregation_func(NN_out[i, :n], dim = 0) # Aggregates only valid instances
            
        output = self.afterNN(output)
        
        return output
    
    
    
def create_3d_data(instances, n_instances):
    ''' Create 3d tensor of data from 2d sequence of instances '''
    max_n_instances = max(n_instances)
    n_bags = len(n_instances)
    n_features = instances.shape[1]
    
    # Pre-allocate empty 3d tensor
    data = torch.empty(n_bags, max_n_instances, n_features)
    
    # Fill data tensor
    marker = 0
    for i in range(n_bags):
        data[i] = torch.cat([ torch.tensor(instances[ marker : marker + n_instances[i] ]) ,  torch.zeros(max_n_instances - n_instances[i], n_features, dtype = torch.double) ], dim = 0)
        marker += n_instances[i]
    
    return data

def ids2n_instances(ids):
    unique, inverse, counts = torch.unique(ids, sorted = True, return_inverse = True, return_counts = True)
    idx = torch.cat([(inverse == x).nonzero()[0] for x in range(len(unique))]).sort()[1]
    bags = unique[idx]
    counts = counts[idx]
    
    return counts

def data_split_3d(data, labels, n_instances, shuffle, test_size = 0.2):
    data = numpy.array(data)
    labels = numpy.array(labels)
    
    X_train, X_test, y_train, y_test, n_instances, n_instances_t = \
        model_selection.train_test_split(data, labels, n_instances, test_size = test_size, shuffle = shuffle)
    
    y = torch.from_numpy(y_train)
    x = torch.from_numpy(X_train)
    x_t = torch.from_numpy(X_test)
    y_t = torch.from_numpy(y_test)
    
    return x, x_t, y, y_t, n_instances, n_instances_t

def accuracy(pred, target, threshold = 0):
    '''
    '''

    pred = pred.detach().numpy()
    target = target.detach().numpy()

    pred[pred >= threshold] = 1
    pred[pred < threshold] = -1

    return numpy.sum(target == pred)/target.shape[0]

def eer(pred, labels):
    fpr, tpr, threshold = metrics.roc_curve(labels.detach(), pred.detach(), pos_label=1)
    fnr = 1 - tpr
    EER_fpr = fpr[numpy.nanargmin(numpy.absolute((fnr - fpr)))]
    EER_fnr = fnr[numpy.nanargmin(numpy.absolute((fnr - fpr)))]
    return EER_fpr, EER_fnr

In [269]:
# Pre and after agg function
prepNN1 = torch.nn.Sequential(
    torch.nn.Linear(5, 5, bias = True),
    torch.nn.ReLU(),
).double()

afterNN1 = torch.nn.Sequential(
    torch.nn.Linear(5, 1, bias = True),
    torch.nn.Tanh()
).double()

In [270]:
import sys
sys.path.append('/Users/kuba/code/aic/mil')
from sklearn.datasets import make_classification
from mill_python.create_dataset.create_bags_simple import create_bags_simple
import numpy
from sklearn import model_selection

# Create data
source_data, source_labels = make_classification(n_samples = 1000, n_features = 5, n_informative = 5, n_redundant = 0, n_repeated = 0, n_classes = 10, class_sep = 1.0, n_clusters_per_class = 1)
data, ids, labels = create_bags_simple(source_data, source_labels, pos = 10, neg = 10, max_instances = 5)
n_instances = ids2n_instances(torch.Tensor(ids))
data_3d = create_3d_data(instances = data, n_instances = n_instances)

# Split data on train and test sets
labels = numpy.array(labels)
data_3d = numpy.array(data_3d)
n_instances = numpy.array(n_instances.float())
x, x_t, y, y_t, n_instances, n_instances_t = data_split_3d(data_3d, labels, n_instances, shuffle = True)

# # Mask identifying valid instances
# mask = torch.zeros(size = (len(n_instances), max(n_instances), len(data[0])))
# for i, n in enumerate(n_instances):
#     mask[i][:n] = 1
    
# Init model
model = BagModel_3d(prepNN1, afterNN1, torch.mean)
model = model.double()

# Criterion and optimizer
import mill_python.src.mil_pytorch as mil

criterion = mil.MyHingeLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3, weight_decay = 1e-3)

In [271]:
# Training
print_loss = True

x = x.double()
n_instances = torch.Tensor(n_instances).long()

for epoch in range(2000):
    pred = model(x, n_instances)
    loss = criterion(pred[:,0], y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Print loss every ** epochs
    if print_loss and ((epoch+1)%50 == 0):
        print('step: {:3d} loss: {}'.format(epoch+1, loss))

step:  50 loss: 0.6791547673166007
step: 100 loss: 0.5880248398068132
step: 150 loss: 0.5318041447066791
step: 200 loss: 0.49816814553246713
step: 250 loss: 0.47753366916092216
step: 300 loss: 0.464933341360306
step: 350 loss: 0.4569306426651829
step: 400 loss: 0.45160790962241515
step: 450 loss: 0.4481721357412302
step: 500 loss: 0.4458142151876857
step: 550 loss: 0.4438326964526862
step: 600 loss: 0.442503700835299
step: 650 loss: 0.44158862594398074
step: 700 loss: 0.4409173565702567
step: 750 loss: 0.4404101642305027
step: 800 loss: 0.4400244221244419
step: 850 loss: 0.439724144512773
step: 900 loss: 0.4394858942831389
step: 950 loss: 0.43929384461586035
step: 1000 loss: 0.43913695867160285
step: 1050 loss: 0.4390073317013165
step: 1100 loss: 0.4388991720195094
step: 1150 loss: 0.4388081514943746
step: 1200 loss: 0.4387309788264032
step: 1250 loss: 0.43866511185935503
step: 1300 loss: 0.438608564845248
step: 1350 loss: 0.43855976999416957
step: 1400 loss: 0.43851729522166794
step: 

In [275]:
# Train set accuracy
pred = model(x, n_instances) 
loss = criterion(pred[:,0], y)
eer_fpr, eer_fnr = eer(pred[:,0], y)
print('loss_train: {}'.format(loss))
print('acc_train: {}'.format(accuracy(pred[:,0], y)))
print('eer_fpr_train: {}'.format(eer_fpr))
print('eer_fnr_train: {}'.format(eer_fnr))

x_t = x_t.double()
n_instances_t = torch.Tensor(n_instances_t).long()

# Test set accuracy
pred = model(x_t, n_instances_t)
loss = criterion(pred[:,0], y_t)
eer_fpr, eer_fnr = eer(pred[:,0], y_t)
print('loss_test: {}'.format(loss))
print('acc_test: {}'.format(accuracy(pred[:,0], y_t)))
print('eer_fpr_test: {}'.format(eer_fpr))
print('eer_fnr_test: {}'.format(eer_fnr))

loss_train: 0.4382660398383135
acc_train: 0.5625
eer_fpr_train: 0.2857142857142857
eer_fnr_train: 0.33333333333333337
loss_test: 0.7500261973394825
acc_test: 0.25
eer_fpr_test: 0.0
eer_fnr_test: 0.0
